<a href="https://colab.research.google.com/github/abdipourasl/OCD/blob/main/MNE_Preprocessing_%26_Connectivity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [122]:
import tensorflow as tf
import os
import cv2
import imghdr
from google.colab import drive
%matplotlib inline
import os
import os.path as op
!pip install mne
import mne
import numpy as np
import pandas as pd
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/')
data_dir= '/content/drive/My Drive/'
!pip install -U mne-connectivity
import mne_connectivity
!pip install asrpy -q
from asrpy import ASR

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<div class="alert alert-block alert-success">
<h1>EEG Preprocessing with MNE</h1>
</div>

### 1. Importing Raw  Data

In [123]:
os.chdir('/content/drive/My Drive/data')
os.listdir('/content/drive/My Drive/data')


['NC', 'OCD']

In [124]:
data_dir= '/content/drive/My Drive/data'
data_dir1 = os.path.join(data_dir,'OCD')
data_dir2 = os.path.join(data_dir,'NC')
list1 = os.listdir(data_dir1)
list2 = os.listdir(data_dir2)
list1

['OCD_subj16.EDF',
 'OCD_subj15.EDF',
 'OCD_subj14.EDF',
 'OCD_subj13.EDF',
 'OCD_subj12.EDF',
 'OCD_subj11.EDF',
 'OCD_subj10.EDF',
 'OCD_subj1.EDF',
 'OCD_subj24.EDF',
 'OCD_subj23.EDF',
 'OCD_subj22.EDF',
 'OCD_subj21.EDF',
 'OCD_subj20.EDF',
 'OCD_subj2.EDF',
 'OCD_subj19.EDF',
 'OCD_subj18.EDF',
 'OCD_subj17.EDF',
 'OCD_subj33.EDF',
 'OCD_subj32.EDF',
 'OCD_subj31.EDF',
 'OCD_subj30.EDF',
 'OCD_subj3.EDF',
 'OCD_subj29.EDF',
 'OCD_subj28.EDF',
 'OCD_subj27.EDF',
 'OCD_subj26.EDF',
 'OCD_subj25.EDF',
 'OCD_subj41.EDF',
 'OCD_subj40.EDF',
 'OCD_subj4.EDF',
 'OCD_subj39.EDF',
 'OCD_subj38.EDF',
 'OCD_subj37.EDF',
 'OCD_subj36.EDF',
 'OCD_subj35.EDF',
 'OCD_subj34.EDF',
 'OCD_subj45.EDF',
 'OCD_subj44.EDF',
 'OCD_subj43.EDF',
 'OCD_subj42.EDF',
 'OCD_subj8.EDF',
 'OCD_subj7.EDF',
 'OCD_subj6.EDF',
 'OCD_subj5.EDF',
 'OCD_subj46.EDF',
 'OCD_subj9.EDF']

In [125]:
raw = mne.io.read_raw_edf(os.path.join('/content/drive/My Drive/','OCD_subj46.EDF'))                                                         # Loading continuous data
print(raw)

Extracting EDF parameters from /content/drive/My Drive/OCD_subj46.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<RawEDF | OCD_subj46.EDF, 23 x 165500 (331.0 s), ~27 kB, data not loaded>


In [ ]:
for i in range(1,2):
  name_dire = f'OCD_subj{i}.EDF'
  raw = mne.io.read_raw_edf(os.path.join(data_dir1,name_dire))
  raw.crop(tmin=5., tmax=50.)
  raw.resample(250, npad="auto")    # set sampling frequency to 256 points per second
  raw.pick_channels(['Fp1','Fp2','F7','F3','F4','F8','T3','C3','C4','T4','T5','P3','P4','T6','O1','O2','Fz','Cz','Pz'],ordered=True)
  print(raw.info["ch_names"])
  print(raw.info["ch_names"])
  raw.set_eeg_reference('average', projection=True).apply_proj()  # re-referencing with the virtual average reference
  asr = ASR(sfreq=raw.info["sfreq"], cutoff=5)
  asr.fit(raw)
  raw._data = asr.transform(raw)
  raw._data.filter(1, 30, fir_design='firwin', picks=['eeg'])  # band-pass filter from 1 to 30 frequency over just
                                                       # EEG channel and not EEG channel
  raw._data.notch_filter(freqs=[16.667, 50]); # bandstop the train and power grid
  raw._data.plot_psd(fmax=100);
  epochs = mne.make_fixed_length_epochs(raw._data, duration=4.0, preload=False, reject_by_annotation=False, proj=False, overlap=2, id=1, verbose=None)
  event_related_plot = epochs.plot_image()
  epochs.get_data()
  #alpha_plv=mne_connectivity.spectral_connectivity_time(epochs.get_data(), freqs=None, method='plv', average=False, indices=None, sfreq=raw.info["sfreq"], fmin=8., fmax=12., fskip=0, faverage=False, sm_times=0, sm_freqs=1)
  #print(alpha_plv.shape)
  #type(alpha_plv)
  con = np.zeros((21, 361, 7))
  for x in range(21):
    a = mne_connectivity.spectral_connectivity_epochs(epochs[x], names=None, method='plv', indices=None, sfreq=raw.info["sfreq"], mode='multitaper', fmin=(0.5,4.,8.,12.,20.,30.,1.), fmax=(4.,8.,12.,20.,30.,40,40.), fskip=0, faverage=True, tmin=0.0, tmax=None, mt_bandwidth=None, mt_adaptive=False, mt_low_bias=True, cwt_freqs=None, cwt_n_cycles=7, block_size=1000, n_jobs=1, verbose=None)
    b = mne_connectivity.EpochSpectralConnectivity.get_data(a)


    con[x, :, :] = b

  #print(con.shape)
  #type(con)


In [138]:
print(con.shape)
type(con)

(21, 361, 7)


numpy.ndarray

In [ ]:
#alpha_plv=mne_connectivity.spectral_connectivity_time(epochs.get_data(), freqs=None, method='plv', average=False, indices=None, sfreq=raw.info["sfreq"], fmin=8., fmax=12., fskip=0, faverage=False, sm_times=0, sm_freqs=1)
#print(alpha_plv.shape)
#type(alpha_plv)
con = np.zeros((21, 361, 7))
for x in range(21):
  a = mne_connectivity.spectral_connectivity_epochs(epochs[x], names=None, method='plv', indices=None, sfreq=raw.info["sfreq"], mode='multitaper', fmin=(0.5,4.,8.,12.,20.,30.,1.), fmax=(4.,8.,12.,20.,30.,40,40.), fskip=0, faverage=True, tmin=0.0, tmax=None, mt_bandwidth=None, mt_adaptive=False, mt_low_bias=True, cwt_freqs=None, cwt_n_cycles=7, block_size=1000, n_jobs=1, verbose=None)
  b = mne_connectivity.EpochSpectralConnectivity.get_data(a)

  print(x)
  print(b.shape)
  con[0, :, :] = b

print(con.shape)
type(con)


In [ ]:
#alpha_plv = mne_connectivity.EpochSpectralConnectivity.get_data(alpha_plv)
print(con.shape)
type(con)

3rd Dimention:
1: Delta
2: Theta
3: Alpha
4: Beta1
5: Beta2
6: Gamma
7: Full Band

In [ ]:
alpha_plv2 = alpha_plv.mean(1)
print(alpha_plv2.shape)
type(alpha_plv2)